In [ ]:
from nbtemplate import display_header, get_path, display_codetoggle
display_header('Chirp.ipynb')

# The need for chirped gratings

In [ ]:
from marxslynx.lynx import PerfectLynx, conf_chirp

In [ ]:
import numpy as np
from astropy.coordinates import SkyCoord
from marxs.source import PointSource, FixedPointing
from marxs.simulator import Sequence

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import copy
conf_nochirp = copy.copy(conf_chirp)
del conf_nochirp['chirp_energy']
del conf_nochirp['chirp_order']

In [ ]:
#ins = PerfectLynx(conf)
#ins5050 = PerfectLynx(conf_5050)
insnochirp = PerfectLynx(conf_nochirp)
inschrip = PerfectLynx(conf_chirp)


In [ ]:
conf_chirp

In [ ]:
from marxslynx.bendgratings import NumericalChirpFinder, chirp_gratings
opt = NumericalChirpFinder(insnochirp.elements[9], insnochirp.elements[2].elements[0], 
                           order=-5.4, energy=0.6, d=0.0002)

In [ ]:
from marxslynx.ralfgrating import facet_table
facetpos = facet_table(insnochirp.elements[2])

In [ ]:
facetz0 = facetpos[np.abs(facetpos['facet_z']) < 50]

In [ ]:
uarray = np.linspace(-.999, .999, 50)
varray = np.array([0])
#gratings = insnochirp.elements[2].elements[facetz0['facet']]
gratings = [insnochirp.elements[2].elements[0], insnochirp.elements[2].elements[319]]
corr = np.zeros((len(gratings), len(uarray)))
for i, grat in enumerate(gratings):
    opt.set_grat(grat)
    corr[i, :] = opt.correction_on_d(uarray, varray)[:, 0]


In [ ]:
corrs = np.stack([e.fitted_d_corr for e in inschrip.elements[2].elements])

In [ ]:
corrsz = corrs[facetz0['facet'], :, 0]

In [ ]:
plt.plot(uarray * 40, corr[0] - 1)
#plt.plot(np.array([-.999, 0, .999]) * 40, corrs[0, :, 0] - 1)
plt.plot(uarray * 40, corr[1] - 1)

In [ ]:
fig = plt.figure(figsize=(4,3))
ax = fig.add_subplot(111)
out = ax.plot(np.array([-.999, 0, .999]) * 40 + 40, corrs[facetz0['facet'], :, 0].T -1, 'k')
ax.set_xlabel('position on grating [mm]')
ax.set_ylabel('fractional change in\ngrating period')
fig.savefig(get_path('figures') + '/chirps.png', dpi=300)
fig.savefig(get_path('figures') + '/chirps.pdf', bbox_inches='tight')

In [ ]:
get_path('figures')

In [ ]:
n_photons = 1e4
instrument = PerfectLynx(conf)

In [ ]:
instrument.elements[2].elements[0]._d

In [ ]:
len(instrument.elements[2].elements)

In [ ]:
mysource = PointSource(coords=SkyCoord(0., 0., unit='deg'),
                           energy=0.5,
                           flux=1.)
fixedpointing = FixedPointing(coords=SkyCoord(0., 0., unit='deg'))
photons = mysource.generate_photons(n_photons)
photons = fixedpointing(photons)
photons = Sequence(elements=instrument.elements[:2])(photons)
p = Sequence(elements=instrument.elements[2:])(photons.copy())

    

In [ ]:
ind = np.isfinite(p['detcirc_phi'])
out = plt.hist(p['detcirc_phi'][ind], weights=p['probability'][ind], bins=100)

In [ ]:
instrument.elements

In [ ]:
from marxslynx.bendgratings import NumericalChirpFinder, chirp_gratings

In [ ]:
instrumchirp = PerfectLynx(conf)
opt = NumericalChirpFinder(instrument.elements[9], instrumchirp.elements[2].elements[0], order=-5, energy=0.5, d=0.0002)

In [ ]:
chirp_gratings(instrumchirp.elements[2].elements, opt)

In [ ]:
from marxs.optics import OrderSelector
from marxs.math.utils import e2h, h2e, norm_vector

In [ ]:
instrumchirp = PerfectLynx(conf)
opt = ChirpFinder(instrument.elements[9], instrumchirp.elements[2].elements[0], order=-5, energy=0.5, d=0.0002)

In [ ]:
epos = np.stack(instrument.elements[2].elem_pos)
plt.scatter(epos[:, 1, 3], epos[:, 2, 3]), c = res[:, 0,0])
plt.colorbar()

In [ ]:
from scipy.interpolate import RectBivariateSpline

In [ ]:
from marxs.analysis.gratings import resolvingpower_from_photonlist

In [ ]:
res0, pos0, std0 = resolvingpower_from_photonlist(p, np.arange(-8, 1), col='detcirc_phi')

In [ ]:
resc, posc, stdc = resolvingpower_from_photonlist(pchirp, np.arange(-8, 1), col='detcirc_phi')

In [ ]:
plt.plot(np.arange(-8, 1), res0)
plt.plot(np.arange(-8, 1), resc)